**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> Here the function act return the next action to do. In a a proportion of `epsilon`, it returns a random action between 0 and 3 (included). Otherwise it learns from the previous state and return an optimized action. It depends on the learned_act function. 

> `epsilon` is essential to avoid getting stuck in a rountine. The learned action can always be the same if we find a rule or even if there's no better action given the previous state (at least from what we learned). Hence the epsilon allows to add new state. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50
epochs_test=15

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> The array `position`is used to track the rat's trajectory. It is an array of size `(grid_size x grid_size)` which is equal to 1 where the rat actually is and were (rat's path cells are set to 1) and 0 everywhere else. -1 values are set at the extremity of the map (width 2).

> The array `board` defines the distribution of reward over the grid. It is an array of size `(grid_size x grid_size)`. `board[x, y]` equals 1 if there's a bonus, -0.5 if there's a malus. Otherwise it is set to 0 if there's neither bonus nor malus in cell (x,y). It equals 1 if there's a bonus, -0.5 otherwise. This array evolves over time as `board[x, y]` is set to 0 for each cell (x,y) visited by the rat. In other words, bonus and malus are fixed when initializing the game and can't be added as time grows. Only removed when eaten by the rat.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):

        state = env.reset()
        game_over = False
        win, lose = 0, 0
        
        while not game_over:
            action = agent.learned_act(state)
            prev_state = state
            state, reward, game_over = env.act(action)
            
            if reward > 0:
                win += reward
            elif reward < 0:
                lose -= reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.5/15.0. Average score (-3.5)
Win/lose count 7.5/8.0. Average score (-2.0)
Win/lose count 6.5/14.0. Average score (-3.8333333333333335)
Win/lose count 12.5/17.0. Average score (-4.0)
Win/lose count 13.0/16.0. Average score (-3.8)
Win/lose count 12.0/16.0. Average score (-3.8333333333333335)
Win/lose count 10.0/7.0. Average score (-2.857142857142857)
Win/lose count 14.0/16.0. Average score (-2.75)
Win/lose count 11.0/18.0. Average score (-3.2222222222222223)
Win/lose count 14.0/20.0. Average score (-3.5)
Win/lose count 8.0/15.0. Average score (-3.8181818181818183)
Win/lose count 12.5/19.0. Average score (-4.041666666666667)
Win/lose count 12.5/20.0. Average score (-4.3076923076923075)
Win/lose count 6.5/10.0. Average score (-4.25)
Win/lose count 9.0/14.0. Average score (-4.3)
Final score: -4.3


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer**

**1)** Let $\pi$ be a policy. By definition we have :
\begin{equation}
\begin{array}{lcl}
    Q^{\pi}(s, a) &=& \mathbb{E}_{P^{\pi}} \Bigg[ \displaystyle \sum_{t\geq0}^{\infty} \gamma^t r(s_t, a_t) \Big| s_0=s, a_0=a \Bigg] \\
    & =& \mathbb{E}_{P^{\pi}} \Bigg[r(s,a) + \displaystyle \sum_{t\geq1}^{\infty} \gamma^{t} r(s_t, a_t) \Big| s_0=s, a_0=a \Bigg] \\
    & = & r(s, a) + \gamma \mathbb{E}_{P^{\pi}} \Bigg[\displaystyle \sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) \Big| s_0=s, a_0=a \Bigg]\\
    & = & \displaystyle \sum_{(a',s')}  \mathbb{E}_{P^{\pi}} \Bigg[\sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) \Big| s_0=s, a_0=a, s_1=s', a_1=a' \Bigg] \times \mathbb{P}\Big(s_1=s', a_1=a'\Big|s_0=s, a_0=a\Big) + \gamma \mathbb{E}_{P^{\pi}} \Bigg[\displaystyle \sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) \Big| s_0=s, a_0=a \Bigg]\\
    & = & \displaystyle \sum_{(a',s')}  \mathbb{E}_{P^{\pi}} \Bigg[\sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) \Big| s_1=s', a_1=a' \Bigg] \times \mathbb{P}\Big(s_1=s', a_1=a'\Big|s_0=s, a_0=a\Big) + \gamma \mathbb{E}_{P^{\pi}} \Bigg[\displaystyle \sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) \Big| s_0=s, a_0=a \Bigg]\\
    & = & \displaystyle \sum_{(a',s')} \mathbb{P}\Big(s_1=s', a_1=a'\Big|s_0=s, a_0=a\Big) \times  Q^{\pi}(s', a') + \gamma \mathbb{E}_{P^{\pi}} \Bigg[\displaystyle \sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) \Big| s_0=s, a_0=a \Bigg]\\
    & = & \mathbb{E}_{(s',a')\sim P(\cdot|s,a)} \big[ Q^{\pi}(s', a') \big] + \gamma \mathbb{E}_{P^{\pi}} \Bigg[\displaystyle \sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) \Big| s_0=s, a_0=a \Bigg]\\
\end{array}
\end{equation}
We end up with, by using the linearity of expectation:
$$Q^{\pi}(s, a) = \mathbb{E}_{(s',a')\sim P(\cdot|s,a)} \big[ r(s,a) + \gamma Q^{\pi}(s', a') \big]$$


**2)** $\pi^*(s') = arg\max_{a'} Q^{*}(s',a')$ is deterministic, so :
\begin{align*}
Q^*(s,a)&= E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
        &= E_{s'\sim p(.|s,a)\,and\,a'\sim \pi^*(.|s')}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
        &= E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)\\
\end{align*}


**3)** We have that $Q^*(s,a)=E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)$
So, $Q^*$ is the solution of the equation $F(Q) = 0$ with $$F(Q) = E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q(s',a') - Q(s,a)\right)$$

Minimizing $\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}(\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2})$ will hence lead us to find the optimal Q.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        # Checking max memory condition & deleting oldest element if necessary
        if len(self.memory) >= self.max_memory:
            self.memory.pop(0)
    
    def random_access(self):
        # Accessing random moves in the memory
        idx = np.random.randint(0, len(self.memory))
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    # espilon decay method added for last question
    def set_epsilon(self, e):
        self.epsilon *= e # e: decay percentage
        
    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, 0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            
            # Remembering previous state
            s_, n_s, a_, r_, game_over_ = self.memory.random_access()
            
            # Saving current state
            input_states[i] = s_
            
            # Get current Q(s,.)
            target_q[i] = self.model.predict(np.expand_dims(s_, 0))
            
            # Update Q(s,a)
            if game_over_:
                # No next state, we take only the reward
                target_q[i,a_] = r_
            else:
                # We update target_q 
                target_q[i,a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s, 0)))
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(20, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Flatten())
        model.add(Dense(10, activation='relu'))
        model.add(Dense(4))
        
        optimizer = Adam(lr=lr, beta_1=0.9, beta_2=0.999, amsgrad=False) #Default params for Adam
        model.compile(optimizer, "mse")
        self.model = model

In [12]:
######################################################################
########### GridSearch commented as it is very long to run ###########
######################################################################
# # Training parameters
# lr = [0.001, 0.01, 0.1]
# memory_size = [5000, 10000, 25000]
# batch_size = [32]

# for lrnng in lr:
#     for mmry in memory_size:
#         for btch in batch_size:
#             print('--- Learning rate: %.3f | Memory: %d | Batch: %d ---'%(lrnng, mmry, btch))
#             env = Environment(grid_size=size, max_time=T, temperature=temperature)
#             agent = DQN_FC(size, lr=lrnng, epsilon = 0.1, memory_size=mmry, batch_size = btch)
#             prefix = 'fc_train_lr' + str(int(1/lrnng)) + '_mmry_' + str(mmry) + '_btch_' + str(btch) + '_'
#             train(agent, env, epochs_train, prefix=prefix)
# #             HTML(display_videos(prefix + '10.mp4'))
#             print('')

env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=25000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

# The best performance is achieved with:
# - Learning rate: 0.001 | Memory: 25000 | Batch: 32













Epoch 000/050 | Loss 0.0020 | Win/lose count 4.5/5.0 (-0.5)
Epoch 001/050 | Loss 0.0049 | Win/lose count 4.0/6.0 (-2.0)
Epoch 002/050 | Loss 0.0027 | Win/lose count 6.0/5.0 (1.0)
Epoch 003/050 | Loss 0.0088 | Win/lose count 8.5/4.0 (4.5)
Epoch 004/050 | Loss 0.0035 | Win/lose count 7.0/2.0 (5.0)
Epoch 005/050 | Loss 0.0032 | Win/lose count 9.5/6.0 (3.5)
Epoch 006/050 | Loss 0.0011 | Win/lose count 10.0/1.0 (9.0)
Epoch 007/050 | Loss 0.0038 | Win/lose count 5.0/1.0 (4.0)
Epoch 008/050 | Loss 0.0021 | Win/lose count 10.0/3.0 (7.0)
Epoch 009/050 | Loss 0.0023 | Win/lose count 4.0/2.0 (2.0)
Epoch 010/050 | Loss 0.0038 | Win/lose count 8.5/3.0 (5.5)
Epoch 011/050 | Loss 0.0020 | Win/lose count 9.5/3.0 (6.5)
Epoch 012/050 | Loss 0.0032 | Win/lose count 12.5/1.0 (11.5)
Epoch 013/050 | Loss 0.0029 | Win/lose count 11.0/3.0 (8.0)
Epoch 014/050 | Loss 0.0017 | Win/lose count 11.5/6.0 (5.5)
Epoch 015/050 | Loss 0.0033 | Win/lose count 12.5/1.0 (11.5)
Epoch 016/050 | Loss 0.0045 | Win/

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
              
        model = Sequential()
        model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(32, kernel_size=3, activation="relu"))
        model.add(Flatten())
        model.add(Dense(4, activation="linear"))
        
        optimizer = Adam(lr=lr)
        model.compile(optimizer, "mse")
        self.model = model

In [14]:
######################################################################
########### GridSearch commented as it is very long to run ###########
######################################################################
# Training parameters
# lr = [0.001, 0.01, 0.1]
# memory_size = [5000, 10000, 25000]
# batch_size = [32]

# for lrnng in lr:
#     for mmry in memory_size:
#         for btch in batch_size:
#             print('--- Learning rate: %.3f | Memory: %d | Batch: %d ---'%(lrnng, mmry, btch))
#             env = Environment(grid_size=size, max_time=T, temperature=temperature)
#             agent = DQN_FC(size, lr=lrnng, epsilon = 0.1, memory_size=mmry, batch_size = btch)
#             prefix = 'cnn_train_lr' + str(int(1/lrnng)) + '_mmry_' + str(mmry) + '_btch_' + str(btch) + '_'
#             train(agent, env, epochs_train, prefix=prefix)
# #            HTML(display_videos(prefix + '10.mp4'))
#             print('')

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=5000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

# The best performance is achieved with:
# - Learning rate: 0.001 | Memory: 5000 | Batch: 32 -

Epoch 000/050 | Loss 0.0047 | Win/lose count 7.0/10.0 (-3.0)
Epoch 001/050 | Loss 0.0031 | Win/lose count 11.5/3.0 (8.5)
Epoch 002/050 | Loss 0.0019 | Win/lose count 4.5/4.0 (0.5)
Epoch 003/050 | Loss 0.0014 | Win/lose count 10.5/3.0 (7.5)
Epoch 004/050 | Loss 0.0011 | Win/lose count 6.0/2.0 (4.0)
Epoch 005/050 | Loss 0.0079 | Win/lose count 15.5/2.0 (13.5)
Epoch 006/050 | Loss 0.0021 | Win/lose count 6.0/1.0 (5.0)
Epoch 007/050 | Loss 0.0016 | Win/lose count 19.5/1.0 (18.5)
Epoch 008/050 | Loss 0.0904 | Win/lose count 19.0/6.0 (13.0)
Epoch 009/050 | Loss 0.0020 | Win/lose count 7.0/4.0 (3.0)
Epoch 010/050 | Loss 0.0058 | Win/lose count 10.5/6.0 (4.5)
Epoch 011/050 | Loss 0.0022 | Win/lose count 12.0/3.0 (9.0)
Epoch 012/050 | Loss 0.0015 | Win/lose count 10.0/2.0 (8.0)
Epoch 013/050 | Loss 0.0020 | Win/lose count 16.0/1.0 (15.0)
Epoch 014/050 | Loss 0.0379 | Win/lose count 13.0/6.0 (7.0)
Epoch 015/050 | Loss 0.0432 | Win/lose count 12.0/4.0 (8.0)
Epoch 016/050 | Loss 0.0020 | Win/lose 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
temp = [0.1, 0.3, 0.5, 0.7]
for t in temp:
    print('\n --- Temp: %.1f ---'%t)
    env = Environment(grid_size=size, max_time=T,temperature=0.3)
    agent_cnn = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=25000, batch_size = 32)
    agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

    agent_fc = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=25000, batch_size = 32)
    agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
    print('Test of the CNN')
    test(agent_cnn,env,epochs_test,prefix='cnn_test')
    print('Test of the FC')
    test(agent_fc,env,epochs_test,prefix='fc_test')


 --- Temp: 0.1 ---
Test of the CNN
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 0/0. Average score (0.75)
Win/lose count 0/0. Average score (0.5)
Win/lose count 2.0/0. Average score (0.875)
Win/lose count 6.5/1.0. Average score (1.8)
Win/lose count 7.0/0. Average score (2.6666666666666665)
Win/lose count 10.0/0. Average score (3.7142857142857144)
Win/lose count 1.5/0. Average score (3.4375)
Win/lose count 1.0/0. Average score (3.1666666666666665)
Win/lose count 0.5/0. Average score (2.9)
Win/lose count 5.0/0. Average score (3.090909090909091)
Win/lose count 4.0/0. Average score (3.1666666666666665)
Win/lose count 1.0/0. Average score (3.0)
Win/lose count 10.0/1.0. Average score (3.4285714285714284)
Win/lose count 4.0/0. Average score (3.466666666666667)
Final score: 3.466666666666667
Test of the FC
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0.5/5.0. Average score (-2.0)
Win/lose count 0.5/0. Average score (-1.1666666666666667)
Win/lose count 0.5/0. Average sc

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

**Answer**

>The agent doesn't seem to explore the map well. It often gets stuck in a position, especially for the agent with the fully connected network.

>As the temperature increases the number of positive rewards cells follows, which helps the agent explore the map since more positive cells gets in his visibility, and therefore the final average score gets higher.



***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent, env, epoch, epsilon=0.1, decay_eps=0.99, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    for e in range(epoch):
        #Setting the value of epsilon
        agent.set_epsilon(epsilon)
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            
            # Epsilon-greedy 
            agent.set_epsilon(decay_eps)
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
         # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        #Initializing the board
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1
        
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
    
    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
    
    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        if not train:
            self.epsilon_agent=0#on met epsilon a 0 en test
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
    
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        reward = 0
        
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""
    
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))
        
        malus[bonus>0]=0

        self.board = bonus + malus
        
        self.board = bonus + malus
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

    
# ## use those samples of code:
# #In train explore:
# state, reward, game_over = env.act(action, train=True)

# ## In Environment exploring:
# # You will have to change n_state to 3 because you will use one more layer!
# reward = 0
# if train:
#     reward = -self.malus_position[self.x, self.y]
# self.malus_position[self.x, self.y] = 0.1

# reward = reward + self.board[self.x, self.y]
# # 3 "feature" states instead of 2
# state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                 self.board.reshape(self.grid_size, self.grid_size,1),
#                         self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [19]:
# Parameters
size = 13
T=200
temperature=0.3
epochs_train=100
epochs_test=15

# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=10000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0030 | Win/lose count 5.0/22.80000000000004 (-17.80000000000004)
Epoch 001/100 | Loss 0.0013 | Win/lose count 2.5/21.600000000000044 (-19.100000000000044)
Epoch 002/100 | Loss 0.0050 | Win/lose count 11.0/21.10000000000001 (-10.100000000000009)
Epoch 003/100 | Loss 0.0023 | Win/lose count 10.0/16.499999999999964 (-6.4999999999999645)
Epoch 004/100 | Loss 0.0044 | Win/lose count 9.5/16.69999999999997 (-7.199999999999971)
Epoch 005/100 | Loss 0.0062 | Win/lose count 9.5/18.500000000000007 (-9.000000000000007)
Epoch 006/100 | Loss 0.0427 | Win/lose count 7.0/18.599999999999994 (-11.599999999999994)
Epoch 007/100 | Loss 0.0189 | Win/lose count 11.0/17.499999999999968 (-6.499999999999968)
Epoch 008/100 | Loss 0.0063 | Win/lose count 14.0/16.49999999999997 (-2.4999999999999716)
Epoch 009/100 | Loss 0.0189 | Win/lose count 8.0/16.799999999999972 (-8.799999999999972)
Epoch 010/100 | Loss 0.0035 | Win/lose count 19.0/15.199999999999966 (3.8000000000000345)
Epoch 011/100 |

Epoch 092/100 | Loss 0.0083 | Win/lose count 12.0/16.499999999999968 (-4.499999999999968)
Epoch 093/100 | Loss 0.0058 | Win/lose count 23.5/11.89999999999998 (11.60000000000002)
Epoch 094/100 | Loss 0.0219 | Win/lose count 23.0/11.699999999999976 (11.300000000000024)
Epoch 095/100 | Loss 0.0339 | Win/lose count 22.5/11.299999999999976 (11.200000000000024)
Epoch 096/100 | Loss 0.0068 | Win/lose count 8.5/17.299999999999976 (-8.799999999999976)
Epoch 097/100 | Loss 0.0214 | Win/lose count 22.0/12.199999999999976 (9.800000000000024)
Epoch 098/100 | Loss 0.0448 | Win/lose count 18.0/11.99999999999998 (6.0000000000000195)
Epoch 099/100 | Loss 0.0034 | Win/lose count 17.5/14.799999999999963 (2.7000000000000366)


In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 4.0/0. Average score (4.0)
Win/lose count 15.0/1.0. Average score (9.0)
Win/lose count 8.5/1.0. Average score (8.5)
Win/lose count 13.0/1.0. Average score (9.375)
Win/lose count 1.0/0. Average score (7.7)
Win/lose count 5.5/0. Average score (7.333333333333333)
Win/lose count 8.5/2.0. Average score (7.214285714285714)
Win/lose count 19.0/1.0. Average score (8.5625)
Win/lose count 17.0/0. Average score (9.5)
Win/lose count 19.5/0. Average score (10.5)
Win/lose count 14.5/0. Average score (10.863636363636363)
Win/lose count 18.5/1.0. Average score (11.416666666666666)
Win/lose count 19.5/2.0. Average score (11.884615384615385)
Win/lose count 12.0/0. Average score (11.892857142857142)
Win/lose count 14.0/1.0. Average score (11.966666666666667)
Final score: 11.966666666666667


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***